# 3.2 Índices Avanzados - Clustered, Nonclustered y Columnstore

**Resumen Ejecutivo**: Domina los tres tipos principales de índices (clustered, nonclustered, columnstore) y cuándo usar cada uno para acelerar consultas transaccionales y analíticas sin degradar escrituras.

---

## 🎯 Objetivos de Aprendizaje
- Diferenciar clustered (un solo orden físico) vs. nonclustered (estructuras separadas).
- Diseñar índices compuestos (orden de columnas) para patrones de consulta frecuentes.
- Entender cuándo columnstore (compresión, analítica masiva) supera row-store tradicional.

## 🧰 Prerrequisitos
- Haber completado `01_performance_basico.ipynb`.
- Conocer planes de ejecución (index seek vs. scan).
- Familiaridad con tablas fact vs. dim (OLTP vs. OLAP).

---

## 📚 Conceptos Clave
- **Clustered index**: ordena físicamente las filas; solo uno por tabla; típicamente en PK.
- **Nonclustered index**: estructura separada que apunta a filas; hasta 999 por tabla; para WHERE/JOIN.
- **Columnstore index**: almacena por columna con compresión; ideal para fact tables con consultas analíticas (SUM, AVG sobre millones de filas).
- **Trade-off**: cada índice acelera lecturas pero ralentiza escrituras (INSERT/UPDATE/DELETE).

> ⚠️ Sobre-indexar degrada rendimiento de escritura; diseña índices para patrones reales, no especulativos.

In [ ]:
-- Columnstore Index para consultas analíticas (data warehouse)
-- ⚠️ Comentado porque fact_ventas es pequeña, pero en producción con 100M+ filas:

-- CREATE NONCLUSTERED COLUMNSTORE INDEX idx_ventas_columnstore
-- ON dbo.fact_ventas(fecha, cliente_id, producto_id, cantidad, descuento_pct);

-- Consulta analítica que se beneficia de columnstore
SELECT YEAR(fecha) AS anio, MONTH(fecha) AS mes,
       SUM(cantidad) AS unidades_totales,
       COUNT(DISTINCT cliente_id) AS clientes_unicos
FROM dbo.fact_ventas
GROUP BY YEAR(fecha), MONTH(fecha)
ORDER BY anio, mes;

/*
🚀 Columnstore es ideal cuando:
✅ Consultas agregan millones de filas (SUM, AVG, COUNT)
✅ Lees pocas columnas de muchas filas
✅ Escrituras son batch (INSERT bulk), no transaccionales

❌ NO uses columnstore si:
❌ Consultas buscan filas individuales por PK
❌ UPDATEs frecuentes row-by-row (OLTP)
❌ Tabla tiene < 1M filas (overhead no vale la pena)
*/

In [ ]:
-- Índice compuesto (múltiples columnas) para consultas de dashboard
CREATE NONCLUSTERED INDEX idx_ventas_cliente_fecha
ON dbo.fact_ventas(cliente_id, fecha DESC)
INCLUDE (cantidad, descuento_pct);

-- Esta consulta usará el índice compuesto eficientemente
SELECT fecha, SUM(cantidad) AS unidades
FROM dbo.fact_ventas
WHERE cliente_id = 5
  AND fecha >= '2024-01-01'
GROUP BY fecha
ORDER BY fecha DESC;

/*
🔑 Orden de columnas en índice compuesto importa:
1. Columna más selectiva primero (cliente_id: filtra a 1 de 100 clientes)
2. Columna de rango segundo (fecha: filtra un período)
3. Columnas adicionales en INCLUDE (evita key lookup)

📊 Selectividad = % de filas únicas
- Alta selectividad (cliente_id): 100 valores únicos de 100 filas = 100%
- Baja selectividad (segmento): 3 valores únicos de 100 filas = 3%
Regla: columna más selectiva va primero en el índice
*/

In [ ]:
-- Crear un índice nonclustered en fact_ventas para optimizar filtros por fecha
CREATE NONCLUSTERED INDEX idx_ventas_fecha 
ON dbo.fact_ventas(fecha)
INCLUDE (cliente_id, producto_id, cantidad, descuento_pct);
-- INCLUDE añade columnas sin ordenarlas, evita "key lookup" (ida extra a la tabla)

-- Consulta que se beneficia del índice anterior
SELECT cliente_id, producto_id, cantidad
FROM dbo.fact_ventas
WHERE fecha >= '2024-01-01' AND fecha < '2024-02-01';

/*
💡 El optimizador usará idx_ventas_fecha porque:
1. WHERE fecha está en el índice (key column)
2. Las columnas del SELECT están en INCLUDE (covering index)
3. No necesita ir a la tabla física (key lookup avoided)
*/

## 🟢 Ejercicio Básico
Crea un índice nonclustered en `dim_productos(categoria)` con INCLUDE(nombre, precio_unitario). Luego ejecuta:
```sql
SELECT nombre, precio_unitario FROM dim_productos WHERE categoria = 'Electrónica';
```
Verifica con `SET STATISTICS IO ON` que no hay key lookups.

## 🟠 Ejercicio Intermedio
Diseña un índice compuesto para optimizar esta consulta:
```sql
SELECT producto_id, SUM(cantidad) AS total
FROM fact_ventas
WHERE region_id = 2 AND fecha >= '2024-01-01'
GROUP BY producto_id;
```
**Entrega:** comando CREATE INDEX con justificación del orden de columnas.

## 🔴 Ejercicio Avanzado
Compara el performance de:
1. Table scan (sin índices)
2. Nonclustered index tradicional
3. Columnstore index

Para esta consulta analítica:
```sql
SELECT YEAR(fecha) AS anio, producto_id, SUM(cantidad) AS unidades
FROM fact_ventas
GROUP BY YEAR(fecha), producto_id;
```

Mide logical reads y elapsed time. ¿Cuándo conviene cada tipo?

---

## Errores Comunes

❌ **Índice en cada columna**: cada índice cuesta en INSERT/UPDATE
❌ **INCLUDE con todas las columnas**: el índice crece innecesariamente
❌ **Orden incorrecto en índice compuesto**: filtras por A y B, pero índice es (B, A)
❌ **Columnstore en tablas OLTP**: transacciones row-by-row no se benefician
❌ **No monitorear fragmentación**: índices se fragmentan con el tiempo (rebuild/reorganize)

---

## Estrategia de Indexación (Guía Rápida)

| Tipo de Consulta | Índice Recomendado |
|------------------|-------------------|
| `WHERE id = X` (PK lookup) | Clustered en PK |
| `WHERE fecha BETWEEN ...` | Nonclustered en fecha |
| `WHERE A = X AND B = Y` | Compuesto (A, B) |
| `SUM/AVG en millones de filas` | Columnstore |
| `JOIN frecuente` | Nonclustered en FKs |

**Siguiente:** `03_particiones_basico.ipynb` → dividir tablas grandes por rango

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](01_performance_basico.ipynb) | [Siguiente ➡️](03_particiones_basico.ipynb)
---
